# Importer le modèle Word2Vec

In [2]:
import os
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import gensim
from gensim.models import word2vec

random.seed(1)
np.random.seed(1)

In [133]:
os.chdir('C:/Users/torna/Documents/StatApp/StatApp')
#os.chdir('C:/Users/Kim Antunez/Documents/Projets_autres')
#os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')
#os.chdir('/home/aqlt/Documents/Ensae/StatApp')

nom_dossier = 'ens'
# dim = 50
dim = 300
epoch = 100
window = 4
learning_rate = 0.02
    
chemin1 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed1".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:]) 
model1 = gensim.models.keyedvectors.KeyedVectors.load(chemin1 + "/word2vec.model")
chemin2 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed5".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model2 = gensim.models.keyedvectors.KeyedVectors.load(chemin2 + "/word2vec.model")
chemin3 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed10".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model3 = gensim.models.keyedvectors.KeyedVectors.load(chemin3 + "/word2vec.model")
chemin4 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed15".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model4 = gensim.models.keyedvectors.KeyedVectors.load(chemin4 + "/word2vec.model")
chemin5 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed20".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model5 = gensim.models.keyedvectors.KeyedVectors.load(chemin5 + "/word2vec.model")
chemin6 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed25".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:]) 
model6 = gensim.models.keyedvectors.KeyedVectors.load(chemin6 + "/word2vec.model")

In [134]:
matrice = {word : (model1.wv[word]+model2.wv[word]+model3.wv[word]+model4.wv[word]+model5.wv[word]+model6.wv[word])/6
           for word in model1.wv.index2word}

## On importe les données de train et on calcule le "sentence-embedding"

In [135]:
# Si on veut récup les fichiers
with open("data/sentimental_analysis/corpus_test.file", "rb") as f:
    corpus_test = pickle.load(f)
with open("data/sentimental_analysis/corpus_train.file", "rb") as f:
    corpus_train = pickle.load(f)

In [136]:
def calcul_sentence_embedding(phrase):
    if len(phrase)>0:
        sum1 = sum(matrice[mot] for mot in phrase if mot in model1.wv.index2word)
        sum2 = sum(matrice["lowfrequencyword"] for mot in phrase if mot not in model1.wv.index2word)
        return (sum1+sum2)/len(phrase)
    else:
        return matrice["lowfrequencyword"]

In [137]:
phrases = [[tweet.split(),score] for (score, tweet) in corpus_train]

In [138]:
phrases_emb = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases]

In [139]:
Y_train = [score for (score, embedding) in phrases_emb]
X_train = [embedding for (score, embedding) in phrases_emb]

In [140]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
reg_log = lr.fit(X_train,Y_train)

In [141]:
Y_train_pred = reg_log.predict(X_train)

from sklearn import metrics
# Matrice confusion
cm = metrics.confusion_matrix(Y_train,Y_train_pred)
print(cm)
# Accuracy
acc = metrics.accuracy_score(Y_train,Y_train_pred)
print(acc) 

[[18048  6952]
 [ 7229 17771]]
0.71638


In [142]:
phrases_test = [[tweet.split(),score] for (score, tweet) in corpus_test]
phrases_test_emb = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases_test]
Y_test = [score for (score, embedding) in phrases_test_emb]
X_test = [embedding for (score, embedding) in phrases_test_emb]
Y_test_pred = reg_log.predict(X_test)

In [143]:
# Matrice confusion
cm_test = metrics.confusion_matrix(Y_test,Y_test_pred)
print(cm_test)
# Accuracy
acc_test = metrics.accuracy_score(Y_test,Y_test_pred)
print(acc_test) 

[[1438  546]
 [ 633 1365]]
0.703917629331994


## Pourquoi ce score ?

In [49]:
phrases = [tweet.split() for (score, tweet) in corpus_train]
mots = [item for sublist in phrases for item in sublist]
vocabulaire = list(dict.fromkeys(mots))
n = 0
absents = []
for word in vocabulaire:
    if word not in model1.wv.index2word:
        n+=1
        absents.append(word)
print(n)
print(len(vocabulaire))

21204
39650


In [50]:
n/len(vocabulaire)

0.5347793190416141

## Stopwords

In [144]:
import nltk
from nltk.corpus import stopwords
stop_words =  set(stopwords.words('french'))

In [145]:
def suppr_stop(phrase):
    return [mot for mot in phrase if mot not in stop_words]
    
phrases_stop = [[suppr_stop(tweet.split()),score] for (score, tweet) in corpus_train]

In [146]:
print(phrases[0])
print(phrases_stop[0])

[['est', 'très', 'heureux', 'que', 'aujourdhui', 'marque', '6', 'merveilleux', 'mois', 'avec', 'stephanie'], 1]
[['très', 'heureux', 'aujourdhui', 'marque', '6', 'merveilleux', 'mois', 'stephanie'], 1]


In [147]:
phrases_emb_stop = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases_stop]
Y_train_stop = [score for (score, embedding) in phrases_emb_stop]
X_train_stop = [embedding for (score, embedding) in phrases_emb_stop]

In [148]:
lr_stop = LogisticRegression()
reg_log_st = lr_stop.fit(X_train_stop,Y_train_stop)

In [149]:
Y_train_pred_stop = reg_log_st.predict(X_train_stop)

# Matrice confusion
cm_stop = metrics.confusion_matrix(Y_train_stop,Y_train_pred_stop)
print(cm_stop)
# Accuracy
acc_stop = metrics.accuracy_score(Y_train_stop,Y_train_pred_stop)
print(acc_stop) 


phrases_test_stop = [[suppr_stop(tweet.split()),score] for (score, tweet) in corpus_test]
phrases_test_emb_stop = [[score,calcul_sentence_embedding_stop(phrase)] for (phrase,score) in phrases_test_stop]
Y_test_stop = [score for (score, embedding) in phrases_test_emb_stop]
X_test_stop = [embedding for (score, embedding) in phrases_test_emb_stop]
Y_test_pred_stop = reg_log_st.predict(X_test_stop)

# Matrice confusion
cm_test_stop = metrics.confusion_matrix(Y_test_stop,Y_test_pred_stop)
print(cm_test_stop)
# Accuracy
acc_test_stop = metrics.accuracy_score(Y_test_stop,Y_test_pred_stop)
print(acc_test_stop) 

[[17455  7545]
 [ 7415 17585]]
0.7008
[[1388  596]
 [ 617 1381]]
0.6953792064289301


In [150]:
# On sauve les modèles
with open("data/sentimental_analysis/reg_log_300.file", "wb") as f:
    pickle.dump(reg_log, f, pickle.HIGHEST_PROTOCOL)
with open("data/sentimental_analysis/reg_log_st_300.file", "wb") as f:
    pickle.dump(reg_log_st, f, pickle.HIGHEST_PROTOCOL)